In [2]:
import pygame
from pygame.locals import *
import time
import random

pygame 2.5.2 (SDL 2.28.3, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [6]:
SIZE = 40
BACKGROUND_COLOR = (110, 110, 5)
class Apple:
    def __init__(self,parent_screen):
        self.image = pygame.image.load("apple.jpg").convert()
        self.parent_screen = parent_screen
        self.x = SIZE*3
        self.y = SIZE*3

    def draw(self):
        self.parent_screen.blit(self.image, (self.x,self.y))
        pygame.display.flip()

    def move(self):
        self.x = random.randint(1,24)*SIZE
        self.y = random.randint(1,19)*SIZE
        

class Snake:
    def __init__(self,parent_screen,length):
        self.parent_screen = parent_screen
        self.block = pygame.image.load("block.jpg").convert()
        self.direction = 'down'
        self.previous_direction = 'down'
        self.length = length
        self.x = [SIZE]*length
        self.y = [SIZE]*length

    def move_left(self):
        if self.direction != 'right':
            self.direction = 'left'
        
    def move_right(self):
        if self.direction != 'left':
            self.direction = 'right'
        
    def move_up(self):
        if self.direction != 'down':
            self.direction = 'up'

    def move_down(self):
        if self.direction != 'up':
            self.direction = 'down'
        
    def walk(self):

        for i in range(self.length-1,0,-1):
            self.x[i] = self.x[i-1]
            self.y[i] = self.y[i-1]
        if self.direction == 'up':
            self.y[0] -= SIZE
        if self.direction == 'down':
            self.y[0] += SIZE
        if self.direction == 'left':
            self.x[0] -= SIZE
        if self.direction == 'right':
            self.x[0] += SIZE
        self.previous_direction = self.direction
        self.draw()

    def draw(self):
        for i in range(self.length):
            self.parent_screen.blit(self.block, (self.x[i],self.y[i]))
        pygame.display.flip()    


    def increase_length(self):
        self.length += 1
        self.x.append(-1)
        self.y.append(-1)
        
    
        
                
        
class Game:
    def __init__(self):
        pygame.init()
        pygame.display.set_caption("ADN Snake And Apple Game")
        
        pygame.mixer.init()
        self.play_background_music()
        self.surface = pygame.display.set_mode((1000,800))
        self.snake =  Snake(self.surface,1)
        self.snake.draw()
        self.apple = Apple(self.surface)
        self.apple.draw()
        
    def is_collission(self, x1, y1, x2, y2):
        if x1 >= x2 and x1 < x2 + SIZE:
            if y1 >= y2 and y1 < y2 + SIZE:
                return True 
        return False

    def play_background_music(self):
        pygame.mixer.music.load("bg_music_1.mp3")
        pygame.mixer.music.play()

    def display_score(self):
        font = pygame.font.SysFont('arial',30)
        score = font.render(f"Score: {self.snake.length}", True, (255,255,255))
        self.surface.blit(score,(800,10))

    def play_sound(self,sound):
        sound = pygame.mixer.Sound(f"{sound}")
        pygame.mixer.Sound.play(sound)

    def render_background(self):
        bg = pygame.image.load("background.jpg")
        self.surface.blit(bg,(0,0))
        
    def play(self):
        self.render_background()
        self.snake.walk()
        self.apple.draw()
        self.display_score()
        pygame.display.flip()

        # snake eating apple
        if self.is_collission(self.snake.x[0], self.snake.y[0], self.apple.x, self.apple.y):
            self.play_sound("1_snake_game_resources_ding.mp3")
            self.snake.increase_length()
            self.apple.move()

        #snake coliding with itself
        for i in range(2,self.snake.length):
            if self.is_collission(self.snake.x[0], self.snake.y[0], self.snake.x[i], self.snake.y[i]):
                self.play_sound("1_snake_game_resources_crash.mp3")
                raise "Collision Occured"
        #snake colliding the wall
        if not (0 <= self.snake.x[0] < 1000 and 0 <= self.snake.y[0] < 800):
            self.play_sound("1_snake_game_resources_crash.mp3")
            raise Exception("Hit the wall")

    def show_game_over(self):
        self.render_background()
        font = pygame.font.SysFont('arial',30)
        line1 = font.render(f"Game is over! Your score is {self.snake.length}", True, (255,255,255))
        self.surface.blit(line1,(200,300))
        line2 = font.render("To play again press Enter. To exit press Escape", True, (255,255,255))
        self.surface.blit(line2, (200,350))
        pygame.display.flip()

        pygame.mixer.music.pause()
        
    def reset(self):
        self.snake =  Snake(self.surface,1)
        self.apple = Apple(self.surface)
               
    def run(self):
        running = True
        pause = False
        speed = 0.2
    
        while running:
            for event in pygame.event.get():
                if event.type == KEYDOWN:
                    if event.key == K_ESCAPE:
                        running = False
                    if event.key == K_RETURN:
                        pygame.mixer.music.unpause()
                        pause = False
                    
                    if not pause:
                        if event.key == K_UP:
                            self.snake.move_up()
                        if event.key == K_DOWN:
                            self.snake.move_down()    
                        if event.key == K_LEFT:
                            self.snake.move_left()
                        if event.key == K_RIGHT:
                            self.snake.move_right()
                        
                elif event.type == QUIT:
                    running  = False

            try:
                if not pause:
                    self.play()
            except Exception as e:
                self.show_game_over()
                pause = True
                self.reset()
            
            time.sleep(speed)
            if not pause:
                if speed > 0.05:
                    speed -= 0.001
        pygame.quit()

                    


if __name__ == "__main__":
    game = Game()
    game.run()





    

<Surface(40x40x24 SW)>